# Proof of concept validation notebook for LEPR upload

In [1]:
import pandas as pd
import xlrd

In [2]:
upload_data = pd.read_excel('../data/upload_validation.xlsx', sheet_name=None)

In [3]:
def extract_chem_dat(upload_data):
    run_products = upload_data['6 Run Products']
    
    header_row_num = 4
    chem_dat_col_index = 13
    run_names = run_products.iloc[header_row_num+1:,0]

    dat = run_products.iloc[:,chem_dat_col_index:]
    dat.columns = dat.iloc[0]
    dat = dat.iloc[1:]
    chem_dat_info = dat.iloc[:2]
    chem_dat_info.index = ['method_id','unit']

    chem_dat = dat.iloc[header_row_num:]
    chem_dat
    chem_dat.index = run_names

    return chem_dat, chem_dat_info

In [4]:
chem_dat, chem_dat_info = extract_chem_dat(upload_data)


In [5]:
chem_dat

,SiO2,SiO2error,TiO2,FeO*,FeOtot
Analytical Data,,,,,
Run_1,47.5,(1),0.61,NaN,NaN
Run_2,49.2,0.1,0.42,NaN,NaN
Run_3,48.1,NaN,0,NaN,NaN
Run_4,46,NaN,<0.7,NaN,NaN
Run_5,49,NaN,nd,NaN,NaN
Run_6,41.3,NaN,-,NaN,NaN
Run_7,41.667688,NaN,0.7,NaN,NaN
Run_8,45.4,NaN,0.9,NaN,NaN
Run_9,46,NaN,0.74,NaN,NaN


In [9]:
def read_formatted_excel(file_path, n):
    # Open the workbook
    # workbook = xlrd.open_workbook(file_path, engine = 'openpyxl')
    workbook = xlrd.open_workbook(file_path)
    # Select the n-th sheet, with 0 as the first shet
    sheet = workbook.sheet_by_index(n)

    # Get the number of rows and columns
    nrows, ncols = sheet.nrows, sheet.ncols

    # Create an empty DataFrame to store the data
    df = pd.DataFrame(index=range(nrows), columns=range(ncols))

    # Iterate through rows and columns, reading formatted values
    for row in range(nrows):
        for col in range(ncols):
            cell = sheet.cell(row, col)
            cell_value = cell.value
            cell_format = workbook.format_map[cell.xf_index]

            if cell_format.format_str and cell.ctype == xlrd.XL_CELL_NUMBER:
                cell_value = cell_format.format(cell_value)

            df.iat[row, col] = cell_value

    return df

In [10]:
read_formatted_excel('../data/upload_validation.xlsx', 1)

XLRDError: Excel xlsx file; not supported